## E.2 MLE for Gaussian AR(1)-GARCH(1,1)

Fit a Gaussian AR(1)-GARCH(1,1) to the 10-year government bond yield. Use the following procedure:

1. Write a function, called "garch11_variance(alpha_0, alpha_1, beta_1, sigma2_1, epsilon)". It takes the parameters of the variance equation as an input as well as the residuals of the mean equation. The function returns the GARCH(1,1) implied variance. Note, the first variance is computed using "epsilon[0]" and the start value of the variance, i.e. "sigma2_1". 

2. Write a second function, called, "Neg_loglikelihood_ar1_Garch11(parameters)". It takes the model parameters as input and returns the negative joint log likelihood function. 

3. Use smart starting values for the optimization (from last week's Python for Financial Data Science material, see below). In addition, we use rather uninformative starting values for beta and sigma2_1, namely 0.01 and 1, respectively. **Praktomat: estimated parameters from local unconstrained optimization**

4. You want to use a bounded optimizer to ensure the estimate imply: (i) stationary interest rates (stationary mean equation), (ii) positive unconditional interest rates, (iii) stationary variance of interest rates (stationary variance equation), (iv) positive unconditional variance of interest rate. **Type of optimizer: differential_evolution**. **Praktomat: estimated parameter global constrained optimization**

5. Hand-in the mathematical algorithm and pseudo code that underlines your Python implementation. 

## Pseudocode for the AR(1)-GARCH(1,1) Model:

Ar model:
\begin{equation}
r_t = \phi_0 + \phi_1 r_{t-1} + epsilon_t
\end{equation}

Garch model:
\begin{equation}
\sigma^2_t = \alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1}, s.t. \sigma^2_1 = \text{known parameter}
\end{equation}

thus the Likelyhood function is as follows:
\begin{equation}
L_T(\phi_0, \phi_1, \alpha_0, \alpha_1, \beta_1, \sigma_1) = \prod_{t=2}^T \frac{1}{\sqrt{ 2 \pi (\alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1})}} \times \exp\left( -\frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} \right)
\end{equation}

and the log Likelihood function:
\begin{equation}
 \ln (L_T(.)) = \sum_{t=2}^T -\frac{1}{2} \ln(2\pi [\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1}]) - \frac{1}{2}  \frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})}
\end{equation}

Instructions:
Compute the Ar model and get the residuals therefrom

Compute the conditional Vraiance: Garch model by using the residuals from the Ar model as well as a starting value
Use an Iterative approach to compute the variance in a "rolling window".

Take the variance and residuals and plug them into the log likelihood function

Use the given optimiziers to find the best Vale for the parameters

further descriptions by comments and code itself


**Data**

In [131]:
# import needed packages
import numpy as np
import pandas as pd
import scipy.optimize

def set_time_index(df, timecolname):

    """This function sets the time col as index and makes sure it's a datetime object.

    :param df: full Dataframe
    :param timecolname: colname of the column that has time information in it
    :return: full Dataframe
    """
    # take the time column and convert it to a datetime object
    df[timecolname] = pd.to_datetime(df[timecolname])

    # set the index of the DF as the time Column
    df.set_index(timecolname, inplace = True)

    return df

In [132]:
# import the needed data
df_bond = pd.read_excel("GovBondYields.xls",sheet_name="Rates",header=0)
df_bond = set_time_index(df_bond,"Date")

# get the 10 year bond
r_t = df_bond.iloc[:,7]

**LL**

\begin{equation}
L_T(\phi_0, \phi_1, \alpha_0, \alpha_1, \beta_1, \sigma_1) = \prod_{t=2}^T \frac{1}{\sqrt{ 2 \pi (\alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1})}} \times \exp\left( -\frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} \right)
\end{equation}

with $\sigma^2_t = \alpha_0 + \alpha_1 \epsilon^2_{t-1} + \beta_1 \sigma^2_{t-1}, s.t. \sigma^2_1 = \text{known parameter}$

Note:
\begin{equation}
 \ln (L_T(.)) = \sum_{t=2}^T -\frac{1}{2} \ln(2\pi [\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1}]) - \frac{1}{2}  \frac{(r_t - [\phi_0 + \phi_1 r_{t-1}])^2}{2 (\alpha_0 + \alpha_1 \epsilon^2_{t-1}+ \beta_1 \sigma^2_{t-1})} 
\end{equation}

**function GARCH_11_VARIANCE**

In [186]:
def garch11_variance_new(alpha_0, alpha_1, beta_1, sigma2_1, epsilon):
    """
    :param alpha_0: param variance equation
    :param alpha_1: param variance equation
    :param beta_1: param variance equation
    :param sigma2_1: param variance equation
    :param epsilon: residuals from mean equation
    :return: implied Garch (1,1) Variance
    """

    # if alpha_0 < 0:
    #     print("alpha0 is negative")
    #     print(alpha_0)

    # if alpha_1 < 0:
    #     print("alpha1 is negative")
    #     print(alpha_1)

    # if beta_1 < 0:
    #     print("beta is negative")
    #     print(beta_1)

    # get the starting value
    sig2_initial = (alpha_0 + alpha_1 * epsilon[0]**2) + beta_1 * sigma2_1

    # get the starting value into a list
    save = [sig2_initial]

    # remove the first epsilon as we already used it to compute the first variance
    epsilon = epsilon[1:]

    # for each epsilon
    for i in range(len(epsilon)):

        # compute the variance by garch formula and take the last value of the array
        sig2 = alpha_0 + alpha_1 * epsilon[i]**2 + beta_1 * save[i]
               # + beta_1 * save[i]

        # save value to array
        save.append(sig2)

    # convert list to numpy array
    save = np.array(save)

    return save


**-ln(L_T)**

In [145]:
def neg_loglikelihood_ar1_Garch11(parameters, r_t):
    """
    :param parameters: list of parameters
    :param r_t: pandas series of wanted infromation
    :return:
    """

    phi_0   = parameters[0]
    phi_1   = parameters[1]
    alpha_0 = parameters[2]
    alpha_1 = parameters[3]
    beta_1 =  parameters[4]
    sig_1 =  parameters[5]

    # get the conditional mean by ar(1) model (remove last row from r_t)
    mean = phi_0 + phi_1 * r_t.iloc[:-1].values

    # compute the residuals from rt (delete first row from r-t)
    resid  = r_t.iloc[1:].values - mean

    # get the variance
    vars = garch11_variance_new(alpha_0, alpha_1, beta_1, sig_1, resid[:-1])

    # compute the likelyhood
    loglikeli = np.sum(-0.5 * np.log(2 * np.pi * vars) - (r_t.iloc[2:].values - mean[1:])**2 / (2 * vars))

    return -loglikeli


**Start Values from 2pass Estimation**

In [187]:
#start values for AR(1)-GARCH(1,1) parameters from last week's Python for Financial Data Science material
phi0_start = 0.0204
phi1_start = 0.9962
alpha0_start = 0.0004
alpha1_start = 0.3157
#uninformative start values for GARCH part
beta1_start = 0.01
sigma2_1_start = 1

# example call
ar1_garch11_params_start = [phi0_start,phi1_start,alpha0_start,alpha1_start,beta1_start,sigma2_1_start]
print(neg_loglikelihood_ar1_Garch11(ar1_garch11_params_start,r_t))

---
4290.227857000036


**Unconstrained Optimization: Nelder-Mead Optimization**

In [193]:
# lokal optimazation of the function
print(scipy.optimize.minimize(neg_loglikelihood_ar1_Garch11,ar1_garch11_params_start,args=(r_t,),method= "Nelder-Mead",options={'ftol' : 1e-12}))


-


<ipython-input-145-6483aba02dda>:15: RuntimeWarning: invalid value encountered in log
  loglikeli = np.sum(-0.5 * np.log(2 * np.pi * vars_) - (r_t.iloc[2:].values - means[1:])**2 / (2 * vars_))


 final_simplex: (array([[-9.42909819e-02,  1.02272100e+00,  6.72904750e-03,
         4.44254154e+00,  3.78591364e-03, -1.89650414e+01],
       [-9.43678041e-02,  1.02273338e+00,  6.72870640e-03,
         4.44201946e+00,  3.78959166e-03, -1.89587335e+01],
       [-9.43805065e-02,  1.02273580e+00,  6.72509682e-03,
         4.43950248e+00,  3.79069163e-03, -1.89425096e+01],
       [-9.43291515e-02,  1.02272597e+00,  6.72679589e-03,
         4.44086270e+00,  3.78849881e-03, -1.89530582e+01],
       [-9.42825107e-02,  1.02272182e+00,  6.72645611e-03,
         4.44080260e+00,  3.78555321e-03, -1.89545001e+01],
       [-9.43404076e-02,  1.02272532e+00,  6.72861485e-03,
         4.44206732e+00,  3.78901032e-03, -1.89601350e+01],
       [-9.43890910e-02,  1.02273693e+00,  6.72550876e-03,
         4.43975714e+00,  3.79120993e-03, -1.89437603e+01]]), array([166.84007587, 166.84008612, 166.84009625, 166.84014612,
       166.84014761, 166.840153  , 166.84016957]))
           fun: 166.84007586992263

**Output:**

**Stationary GARCH(1,1)** Stationary Conditions and Positivity Restrictions for the Variance:

Stationary mean equation:
\begin{equation}
|\phi_1| < 1
\end{equation}

Stationary variance equation:
\begin{equation}
\alpha_1 + \beta_1 < 1
\end{equation}

Positive unconditional variance forecast:
\begin{equation}
\alpha_0 > 0 \qquad \text{and} \qquad \alpha_1, \beta_1, \sigma^2_{1} \in \mathcal{R}_+
\end{equation}

Positive unconditional interest rates:
\begin{equation}
\phi_0 > 0, \qquad \phi_1 > 0
\end{equation}


**Bounded Optimization:** 

Hints:

1. Please specify the bounds for all the parameters.

2. For inequality constraints please following the doc from scipy in the following link:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html. In addition, for $ \alpha_1 + \beta_1 < 1$ we specify it as $ 0<\alpha_1 + \beta_1 < 1$. 

3. Please use seed=1

In [191]:
# get the length: number of unknown parameters
K = len(ar1_garch11_params_start)

def linearconstraint(params):
    """
    :param params: Params list
    :return: sum of the 2 variables
    """
    return np.array(params[3] + params[4])

# create a linaer constraint for the inequality
constraint = scipy.optimize.NonlinearConstraint(linearconstraint,lb = 0, ub = 1)

# lower bounds
lb = 0.000001 * np.ones(K)

# upper bounds
ub = 1 * np.ones(K)

# create the bounds
bounds = tuple((lb[x],ub[x]) for x in range(0,K))

In [192]:
# optimize globally
optimal_values_globally = scipy.optimize.differential_evolution(func = neg_loglikelihood_ar1_Garch11,args=(r_t,),bounds = bounds,seed = 1,constraints = constraint)


c:\users\win10\pycharmprojects\elasticstack\venv\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


           constr: [array([0.])]
 constr_violation: 0.0
              fun: -76.80268667510582
              jac: [array([[0., 0., 0., 1., 1., 0.]]), array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])]
            maxcv: 0.0
          message: 'Optimization terminated successfully.'
             nfev: 5996
              nit: 93
          success: True
                x: array([4.95223831e-02, 9.91773641e-01, 2.26752586e-04, 1.44879680e-01,
       8.55120292e-01, 2.80554551e-03])
